Imports

In [54]:
import requests
from PIL import Image
import imagehash
import pandas as pd
import ipyplot
import distance                                                    
import numpy                                                                     
import glob
import cv2
import os

Make sure the directory works

In [55]:
f = r'/Users/danielbernardo/Desktop/image_hash_full-Dataset'

In [56]:
os.listdir(f)

['D21_0043_3_1_2.png',
 'D21_0033_3_4_1.png',
 'S29_0023_3_1_2.png',
 'Z1_0005_3_4_1.png',
 'S29_0053_3_4_1.png',
 'I10_0002_1_2_9.png',
 'N37_N35_0001_1_2_9.png',
 'M4_0001_3_4_1.png',
 'L1_0001_3_4_1.png',
 'A1_Q3_0001_1_1_4.png',
 'D36_0007_1_1_4.png',
 'G1_0003_3_4_1.png',
 'N23_0021_3_4_1.png',
 'F4_0003_3_4_1.png',
 'Z1_Z2_V22_0001_1_1_4.png',
 'D58_0006_3_4_1.png',
 'Z7_0015_1_2_9.png',
 'Z7_0009_3_1_2.png',
 'D46_0013_3_4_1.png',
 'A1_0112_3_4_1.png',
 'S29_0011_1_1_2.png',
 'N14_0009_3_4_1.png',
 'D36_X1_0002_3_4_1.png',
 'A1_0010_2_4_1.png',
 'X1_0019_1_1_4.png',
 'X1_D21_0001_1_1_4.png',
 'M26_0004_1_1_2.png',
 'Q3_N35_0002_1_4_11.png',
 'S29_0003_1_4_12.png',
 'E34_N35_U1_Aa11_D36_0001_3_1_2.png',
 'W19_0007_3_4_1.png',
 'N35_0049_1_2_7.png',
 'G43_0074_3_4_1.png',
 'A1_0016_1_1_2.png',
 'G43_0004_3_1_2.png',
 'G17_0034_1_1_4.png',
 'D21_N35_S29_0005_3_1_2.png',
 'M17_0062_3_1_2.png',
 'A1_0024_3_1_2.png',
 'A1_0054_3_4_1.png',
 'M17_0012_3_4_1.png',
 'A1_0038_1_2_9.png',
 

Resize all images:

on mac devices make sure to delete .ds store with 'find . -name '.DS_Store' -type f -delete'

In [57]:
f = r'/Users/danielbernardo/Desktop/image_hash_full-Dataset'
for file in os.listdir(f):
    f_img = f+"/"+file
    img = Image.open(f_img)
    img = img.resize((200,200))
    img.save(f_img)

Function to Hash images for comparison

In [58]:
df = pd.DataFrame(columns=['image','ahash','phash','dhash','whash','colorhash'])

for file in os.listdir(f):
    filename = file
    file = f+"/"+file
    file = Image.open(file)
    

    data = {
        'Image_Name': filename,
        'ahash': imagehash.average_hash(file),
        'phash': imagehash.phash(file),
        'dhash': imagehash.dhash(file),
        'whash': imagehash.whash(file),
        'colorhash': imagehash.colorhash(file),   
    }
    
    df = df.append(data, ignore_index=True)

/var/folders/z3/w4mm5rz15ldfcx5ccrbqzfh00000gn/T/ipykernel_73362/1806343030.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/var/folders/z3/w4mm5rz15ldfcx5ccrbqzfh00000gn/T/ipykernel_73362/1806343030.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/var/folders/z3/w4mm5rz15ldfcx5ccrbqzfh00000gn/T/ipykernel_73362/1806343030.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/var/folders/z3/w4mm5rz15ldfcx5ccrbqzfh00000gn/T/ipykernel_73362/1806343030.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.

In [ ]:
df.head(10)

Funtion to find similar images by comparing hashes with hamming distance and return df to best matches 

In [59]:
def find_similar_images(df, ahash_column, input_image):
    input = Image.open(input_image)
    ahash = str(imagehash.average_hash(input))
        
    df['hamming_distance'] = df.apply(\
    lambda x: distance.hamming(str(x[ahash_column]), ahash), axis=1)

    df = df[['image','ahash','hamming_distance']]\
    .sort_values(by='hamming_distance', ascending=True)
    
    return df

In [60]:
df = find_similar_images(df, 'ahash', '/Users/danielbernardo/Desktop/image_hash_full-Dataset/N35_Aa1_0001_3_1_2.png')

In [61]:
pd.set_option("display.max_columns", 100)

In [62]:
df.head(50)

,image,ahash,hamming_distance
4195,NaN,f0c1bbf0f0f9f3ef,0
13099,NaN,c1c1fbf0f0f9e30f,5
2419,NaN,f9c1800080f9f3ef,5
3993,NaN,e0c1bb60e0fdf3e7,5
10047,NaN,f8c10100f8f9f3ef,5
10748,NaN,f3c10000fdf9f3ef,5
3755,NaN,f0c1c3e0f0f9f307,5
1656,NaN,f1c181f0f0f1c38f,6
6264,NaN,f000fbc01cf9f3ef,6
967,NaN,e3c78100f0f9f3ef,6
